In [1]:
import pickle
import pandas as pd

model_path = open('model/model_regression.pkl', 'rb')
model = pickle.load(model_path)
model

DecisionTreeRegressor()

In [2]:
df = pd.read_csv('data/MSFT_Processed_Data.csv', parse_dates=['Date'], index_col=['Date'])
df.head()

,Open,High,Low,Close,Adj Close,Volume,change_tmr_%,Tmr_Projection
Date,,,,,,,,
2016-12-09,61.180000,61.990002,61.130001,61.970001,56.274208,27349400,0.321694,UP
2016-12-12,61.820000,62.299999,61.720001,62.169998,56.455818,20198100,1.286125,UP
2016-12-13,62.500000,63.419998,62.240002,62.980000,57.191376,35718900,-0.478620,DOWN
2016-12-14,63.000000,63.450001,62.529999,62.680000,56.918961,30352700,-0.159793,DOWN
2016-12-15,62.700001,63.150002,62.299999,62.580002,56.828140,27669900,-0.449442,DOWN


In [3]:
from backtesting import Backtest, Strategy

In [4]:
class Regression(Strategy):
    def init(self):
        self.model = model
        self.already_bought = False
    
    def next(self):
        explanatory_today = self.data.df.tail(1)
        forecast_tmr = model.predict(explanatory_today)[0]
        if forecast_tmr > 5 and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tmr < -5 and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

In [5]:
df_explanatory = df.drop(columns=["change_tmr_%",'Adj Close',	"Tmr_Projection"])

In [6]:
bt = Backtest(df_explanatory, Regression, cash=10000, commission=0.002, exclusive_orders=True)

In [7]:
result = bt.run()

In [8]:
result.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-09 00:00:00
End,2024-09-05 00:00:00
Duration,2827 days 00:00:00
Exposure Time [%],88.591984
Equity Final [$],42012.045519
Equity Peak [$],48185.86577
Return [%],320.120455


In [12]:
bt.plot(filename='backtesting_report/regression.html')

Row(id='4463', ...)